# Step 1

Run this to create the function that translate the MULTIPOLYGON string from geo-network to MultiPolygon array in GeoJson

In [1]:
import re

# original = "MULTILINESTRING((-177.74411010742188 -13.038644790649414,-178.20994567871094 -13.532644271850586,-178.66815185546875 -14.000577926635742,-180 -14.845966826181044),(180 -14.845966826181044,179.51083374023438 -15.151217460632324,179.3552703857422 -15.259571075439453,179.10226440429688 -15.414399147033691,178.74452209472656 -15.648521423339844,177.78695678710938 -16.244556427001953))"

def get_elements(a_str: str) -> list:
    if re.compile(r'\)\),\s*\(\(').search(a_str):
        a_str = re.sub(r'\)\),\s*\(\(', ');;(', a_str)
        elements = a_str.split(';;')
    else:
        elements = []
        elements.append(a_str)
    return elements

def translate_coordinates(element: str) -> list:
    mp = re.sub(r'^\(', '', element)
    mp = re.sub(r'\)$', '', mp)
    coors = mp.split(",")
    ar = []
    for coor in coors:
        ar.append([float(i) for i in coor.strip().split(" ")])
    return ar

def translate_trajectory_multipolygon(elements: list) -> list:
    results = []
    for i, element in enumerate(elements):
        processed_ar = []
        if re.compile(r'\),\s*\(').search(element):
            element = re.sub(r'\),\s*\(', ');(', element)
            sub_elements = element.split(');(')
            for sub_element in sub_elements:
                processed_ar.append(translate_coordinates(sub_element))
        else:
            processed_ar.append(translate_coordinates(element))
        results.append(processed_ar)
    return results

def translate_multilinestring(elements: list) -> list:
    results = []
    if re.compile(r'\),\s*\(').search(elements[0]):
        element = re.sub(r'\),\s*\(', ');(', elements[0])
        sub_elements = element.split(');(')
        for sub_element in sub_elements:
            results.append(translate_coordinates(sub_element))
    else:
        results.append(translate_coordinates(element))
    return results

def translate_linestring(elements: list) -> list:
    return translate_coordinates(elements[0])


def translate_point(elements: list) -> list:
    return translate_coordinates(elements[0])[0]


def translate_geometry(original: str) -> list:
    if re.compile(r'^LINESTRING\s*\(').search(original):
        a_str = re.sub(r'\)$', '', re.sub(r'LINESTRING\s*\(', '', original))
        return 'LineString', translate_linestring(get_elements(a_str))

    if re.compile(r'^MULTILINESTRING\s*\(').search(original):
        a_str = re.sub(r'\)$', '', re.sub(r'MULTILINESTRING\s*\(', '', original))
        elements = get_elements(a_str)
        return 'MultiLineString', translate_multilinestring(elements)
    
    if re.compile(r'^MULTIPOLYGON\s*\(').search(original):
        a_str = re.sub(r'\)\)$', '', re.sub(r'MULTIPOLYGON\s*\(\(', '', original))
        return 'MultiPolygon', translate_trajectory_multipolygon(get_elements(a_str))

    if re.compile(r'^POINT\s*\(').search(original):
        a_str = re.sub(r'POINT\s*', '', original)
        return 'Point', translate_point(get_elements(a_str))


# results = translate_geometry(original)
# results

In [2]:
import psycopg2
from psycopg2 import Error

try:
    # Connect to an existing database
    connection = psycopg2.connect(user="postgres",
                                  password="password123",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="soop_sst")
    # Create a cursor to perform database operations
    cursor = connection.cursor()
    # Print PostgreSQL details
    print("PostgreSQL server information")
    print(connection.get_dsn_parameters(), "\n")
    # Executing a SQL query
    cursor.execute("SELECT version();")
    # Fetch result
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")
except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

PostgreSQL server information
{'user': 'postgres', 'dbname': 'soop_sst', 'host': '127.0.0.1', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 13.8 (Ubuntu 13.8-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0, 64-bit',) 



In [3]:
cursor.execute("SELECT ST_AsText(geom) FROM soop_sst.soop_sst_nrt_trajectory_map WHERE ST_AsText(geom) IS NOT NULL;")
geom_astxt_records = [record[0] for record in cursor.fetchall()]
print(len(geom_astxt_records))

21620


In [4]:
# Some init setting, replace the API key for your instance

import urllib.request as request
import requests as postApi
import json

headers = {
    "Content-Type": "application/json",
    "Authorization": "ApiKey WTlTdUNZVUJUWGtsWTg4N1Z2T1Q6cFFadkFicGdSbFNnbVdPRkNrcWVKdw=="
}


In [5]:
# Create the correct Mapping, the key here is to map the geoPolygon to geo_shape, the elastic do not guess the type correct and
# hence need to override it.
x = postApi.put('https://ec2-54-253-84-18.ap-southeast-2.compute.amazonaws.com:9200/search-soop-sst-postgres', headers=headers, json={
    "settings": {
        "index": {
            "number_of_shards": "1",
            "analysis": {
                "filter": {
                    "front_ngram": {
                        "type": "edge_ngram",
                        "min_gram": "1",
                        "max_gram": "12"
                    },
                    "bigram_joiner": {
                        "max_shingle_size": "2",
                        "token_separator": "",
                        "output_unigrams": "false",
                        "type": "shingle"
                    },
                    "bigram_max_size": {
                        "type": "length",
                        "max": "16",
                        "min": "0"
                    },
                    "en-stem-filter": {
                        "name": "light_english",
                        "type": "stemmer",
                        "language": "light_english"
                    },
                    "bigram_joiner_unigrams": {
                        "max_shingle_size": "2",
                        "token_separator": "",
                        "output_unigrams": "true",
                        "type": "shingle"
                    },
                    "delimiter": {
                        "split_on_numerics": "true",
                        "generate_word_parts": "true",
                        "preserve_original": "false",
                        "catenate_words": "true",
                        "generate_number_parts": "true",
                        "catenate_all": "true",
                        "split_on_case_change": "true",
                        "type": "word_delimiter_graph",
                        "catenate_numbers": "true",
                        "stem_english_possessive": "true"
                    },
                    "en-stop-words-filter": {
                        "type": "stop",
                        "stopwords": "_english_"
                    }
                },
                "analyzer": {
                    "i_prefix": {
                        "filter": [
                            "cjk_width",
                            "lowercase",
                            "asciifolding",
                            "front_ngram"
                        ],
                        "type": "custom",
                        "tokenizer": "standard"
                    },
                    "iq_text_delimiter": {
                        "filter": [
                            "delimiter",
                            "cjk_width",
                            "lowercase",
                            "asciifolding",
                            "en-stop-words-filter",
                            "en-stem-filter"
                        ],
                        "type": "custom",
                        "tokenizer": "whitespace"
                    },
                    "q_prefix": {
                        "filter": [
                            "cjk_width",
                            "lowercase",
                            "asciifolding"
                        ],
                        "type": "custom",
                        "tokenizer": "standard"
                    },
                    "iq_text_base": {
                        "filter": [
                            "cjk_width",
                            "lowercase",
                            "asciifolding",
                            "en-stop-words-filter"
                        ],
                        "type": "custom",
                        "tokenizer": "standard"
                    },
                    "iq_text_stem": {
                        "filter": [
                            "cjk_width",
                            "lowercase",
                            "asciifolding",
                            "en-stop-words-filter",
                            "en-stem-filter"
                        ],
                        "type": "custom",
                        "tokenizer": "standard"
                    },
                    "i_text_bigram": {
                        "filter": [
                            "cjk_width",
                            "lowercase",
                            "asciifolding",
                            "en-stem-filter",
                            "bigram_joiner",
                            "bigram_max_size"
                        ],
                        "type": "custom",
                        "tokenizer": "standard"
                    },
                    "q_text_bigram": {
                        "filter": [
                            "cjk_width",
                            "lowercase",
                            "asciifolding",
                            "en-stem-filter",
                            "bigram_joiner_unigrams",
                            "bigram_max_size"
                        ],
                        "type": "custom",
                        "tokenizer": "standard"
                    }
                }
            },
            "number_of_replicas": "1"
        }
    },
    "mappings": {
        "dynamic": "true",
        "dynamic_templates": [
            {
                "all_text_fields": {
                    "match_mapping_type": "string",
                    "mapping": {
                        "analyzer": "iq_text_base",
                        "fields": {
                            "delimiter": {
                                "analyzer": "iq_text_delimiter",
                                "type": "text",
                                "index_options": "freqs"
                            },
                            "joined": {
                                "search_analyzer": "q_text_bigram",
                                "analyzer": "i_text_bigram",
                                "type": "text",
                                "index_options": "freqs"
                            },
                            "prefix": {
                                "search_analyzer": "q_prefix",
                                "analyzer": "i_prefix",
                                "type": "text",
                                "index_options": "docs"
                            },
                            "enum": {
                                "ignore_above": 2048,
                                "type": "keyword"
                            },
                            "stem": {
                                "analyzer": "iq_text_stem",
                                "type": "text"
                            }
                        }
                    }
                }
            },
            {
                "geo_array_as_shape": {
                    "match_mapping_type": "*",
                    "match": "geoPolygon",
                    "mapping": { "type" : "geo_shape"}
                }
            }
        ]
    }
}, verify=False);

print(x.text)

/home/vhnguyen/anaconda3/envs/architecturereview/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ec2-54-253-84-18.ap-southeast-2.compute.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{"acknowledged":true,"shards_acknowledged":true,"index":"search-soop-sst-postgres"}


In [6]:
from tqdm import tqdm
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# post document to elasticsearch indice
for i in tqdm(range(len(geom_astxt_records))):
    geojson_type, translated_geom = translate_geometry(geom_astxt_records[i])
    data = {
    "geoPolygon":
        {
            "type": geojson_type,
            "coordinates": translated_geom
        }
    }
    postApi.post(
        "https://ec2-54-253-84-18.ap-southeast-2.compute.amazonaws.com:9200/search-soop-sst-postgres/_doc",
        json = data,
        verify=False,
        headers=headers
    )
    
# close database connection
if (connection):
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21620/21620 [1:30:19<00:00,  3.99it/s]

PostgreSQL connection is closed
